
CSYE7105 High Performance Parallel Machine Learning and AI

Instructor: Dr. Handan Liu

Lecture 12: Diagnostics (local and distributed)


In [1]:
import dask.array as da

In [2]:
from dask.diagnostics import ProgressBar
a = da.random.normal(size=(10000, 10000), chunks=(1000, 100))
res = a.dot(a.T).mean(axis=0)

with ProgressBar():
    out = res.compute()

/home/h.liu/.conda/envs/py2025/lib/python3.11/site-packages/dask/array/routines.py:332: PerformanceWarning: Increasing number of chunks by factor of 10
  intermediate = blockwise(


[########################################] | 100% Completed | 115.53 s


In [3]:
from dask.diagnostics import ResourceProfiler
rprof = ResourceProfiler(dt=0.5)

In [4]:
rprof

In [5]:
import dask.array as da
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
a = da.random.random(size=(10000, 1000), chunks=(1000, 1000))
q, r = da.linalg.qr(a)
a2 = q.dot(r)

with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof:
        out = a2.compute()

In [6]:
prof.visualize()

figure(id='p1001', ...)

In [8]:
prof.results[2]

TaskData(key=('random_sample-1aafb0da0fe74ad7eb77e5625143c40b', 1, 0), task=<Task ('random_sample-1aafb0da0fe74ad7eb77e5625143c40b', 1, 0) _apply_random(...)>, start_time=7268688.754695331, end_time=7268688.77673849, worker_id=23192677246528)

In [9]:
rprof.results

[ResourceData(time=7268688.757953593, mem=4447.023104, cpu=0.0),
 ResourceData(time=7268689.018836768, mem=4457.877504, cpu=329.8),
 ResourceData(time=7268689.269191277, mem=4458.405888, cpu=387.4),
 ResourceData(time=7268689.519640702, mem=4464.4352, cpu=387.3),
 ResourceData(time=7268689.770129676, mem=4466.593792, cpu=387.3),
 ResourceData(time=7268690.020535613, mem=4466.597888, cpu=387.3),
 ResourceData(time=7268690.270947232, mem=4466.597888, cpu=383.3),
 ResourceData(time=7268690.521392227, mem=4466.597888, cpu=391.3),
 ResourceData(time=7268690.771794332, mem=4466.860032, cpu=387.4),
 ResourceData(time=7268691.022177838, mem=4527.931392, cpu=387.4),
 ResourceData(time=7268691.272576404, mem=4723.032064, cpu=375.4),
 ResourceData(time=7268691.522991417, mem=4726.677504, cpu=391.4),
 ResourceData(time=7268691.773425212, mem=4791.82848, cpu=387.3),
 ResourceData(time=7268692.023854669, mem=4808.736768, cpu=387.3),
 ResourceData(time=7268692.274321462, mem=4590.825472, cpu=387.3),


In [10]:
cprof.results

[CacheData(key=('random_sample-1aafb0da0fe74ad7eb77e5625143c40b', 8, 0), task=<Task ('random_sample-1aafb0da0fe74ad7eb77e5625143c40b', 8, 0) _apply_random(...)>, metric=1, cache_time=7268688.790813615, free_time=7268689.417619819),
 CacheData(key=('qr-b1896ce4d9c78797a7651d163af1ea5c', 8, 0), task=<Task ('qr-b1896ce4d9c78797a7651d163af1ea5c', 8, 0) _wrapped_qr(...)>, metric=1, cache_time=7268689.417619819, free_time=7268689.418070994),
 CacheData(key=('random_sample-1aafb0da0fe74ad7eb77e5625143c40b', 9, 0), task=<Task ('random_sample-1aafb0da0fe74ad7eb77e5625143c40b', 9, 0) _apply_random(...)>, metric=1, cache_time=7268688.77563648, free_time=7268689.574057883),
 CacheData(key=('qr-b1896ce4d9c78797a7651d163af1ea5c', 9, 0), task=<Task ('qr-b1896ce4d9c78797a7651d163af1ea5c', 9, 0) _wrapped_qr(...)>, metric=1, cache_time=7268689.574057883, free_time=7268689.574379731),
 CacheData(key=('random_sample-1aafb0da0fe74ad7eb77e5625143c40b', 1, 0), task=<Task ('random_sample-1aafb0da0fe74ad7eb77e

In [11]:
cprof.results[0]

CacheData(key=('random_sample-1aafb0da0fe74ad7eb77e5625143c40b', 8, 0), task=<Task ('random_sample-1aafb0da0fe74ad7eb77e5625143c40b', 8, 0) _apply_random(...)>, metric=1, cache_time=7268688.790813615, free_time=7268689.417619819)

In [12]:
from dask.distributed import Client
client = Client(processes=False)
client

/home/h.liu/.conda/envs/py2025/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45637 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.99.249.61:45637/status,
Dashboard: http://10.99.249.61:45637/status,Workers: 1
Total threads: 4,Total memory: 25.00 GiB
Status: running,Using processes: False
Comm: inproc://10.99.249.61/750407/1,Workers: 1
Dashboard: http://10.99.249.61:45637/status,Total threads: 4
Started: Just now,Total memory: 25.00 GiB
Comm: inproc://10.99.249.61/750407/4,Total threads: 4
Dashboard: http://10.99.249.61:45537/status,Memory: 25.00 GiB
Nanny: None,


In [15]:
from dask.distributed import get_task_stream 

a = da.random.random(size=(10000, 1000), chunks=(1000, 1000))
q, r = da.linalg.qr(a)
x = q.dot(r)

with get_task_stream(plot='save', filename="task-stream.html") as ts:
    x.compute()

client.profile(filename="dask-profile.html")

history = ts.data